# Computational Neuroscience Modeling Project
## Half Center Oscillator - Two Cells

### Setup before building the model
Only need to run the three cells below once.
#### Install NEURON
First install NEURON package.

In [ ]:
!pip install neuron

#### Download modfiles from github

In [ ]:
import os

if os.path.basename(os.getcwd())!='HCO-Model-Project-Colab':
  !git clone https://github.com/chenziao/HCO-Model-Project-Colab.git
  %cd HCO-Model-Project-Colab/
else:
  print('Mod files have already been downloaded.')
%ls

#### Compile modfiles

In [ ]:
import os
print(os.system('nrnivmodl')) # compile modfiles. Return 0 for success, 1 for failure.

### Build the HCO model
Now you are ready to build the two-cell HCO model using the same template for both cell.

#### Initialize the model
Run the cell below only once to import the cell template from the script "HCO_cell_template.py" and create two cells (cellA and cellB) using the same template. Then provide the first cell (cellA) with a current injection and create two synapses that mutually connect the two cells.

In [ ]:
from neuron import h
from HCO_cell_template import HCOcell
import matplotlib.pyplot as plt
import ipywidgets as widgets

h.load_file('stdrun.hoc')

h.dt = 0.025 # time step (resolution) of the simulation in ms
h.v_init= -50 # initial membrane potential in mV

# create two cells from HCO_cell_template
cellA = HCOcell()
cellB = HCOcell()
cells = [cellA,cellB] # put the two cells in a list

# create a current clamp to cellA
ccl = h.IClamp(cellA.soma(.5))

# create two synapses connecting the two cells
syn = [] # make empty list for synapse object
nc = [] # make empty list for NetCon object
for cell in cells:
    syn.append(h.inhsyn(.5,sec=cell.soma)) # create a synapse into each cell
for i,cell in enumerate(cells):
    nc.append(h.NetCon(cell.soma(.5)._ref_v,syn[1-i],0,0,10,sec=cell.soma)) # a NEURON object that connects a source to a target

#### Set parameters and simulate
In the code below, you can change values for the parameters listed. The same biophysical and synaptic parameters in the dictionaries are used for both cells. The current injection parameters are used only for cellA.

Run the code after setting up the paremeter values, the simulation will run. After the simulation is done, run the code in the next step and you will see the result plots and the values of the parameters being used printed out.

You can change the parameters and run the code below repeatedly to tune the model.

In [ ]:
h.tstop = 2000 # Simulation time (ms)

# Biophysical parameters
parameters = {
    # Conductance of channels (siemens/cm2)
    'gbar_leak': None,
    'gbar_na': .1,    # (.1~.5)
    'gbar_kdr': .1,    # (.1~.5)
    'gbar_ka': .1,    # (.1~.5)
    'gbar_kca': .01,    # (.01~.05)
    'gbar_cas': .001,    # (.001~.01)
    'gbar_cat': .005,    # (.005~.01)
    'gbar_hyper': .0001,    # (.0001~.0003)
    # Ca pool parameters
    'tauca_capool': None,    # decay time constant
    'fca_capool': None    # ca influx factor that goes to ca pool
} # parameters set to "None" are to use default values defined in the template

# Current injection parameters
ccl_param = {
    'delay': 100,    # start time (ms)
    'dur': 10,    # duration (ms)
    'amp': 1.0    # amplitude (nA) (set to 0 to disable current injection)
}

# Synapse parameters
syn_param = {
    'esyn': -80,    # synaptic channels reversal potential (mV)
    'gmax': 40e-3,    # synaptic channels maximum conductance (uS) (default: 40e-3)
    'tau1': 10,    # rise time (ms) (default: 10)
    'tau2': 20    # decay time (ms) (default: 20)
}


# Setup parameters (Do Not need to change the code below)
for p,v in ccl_param.items():
    setattr(ccl,p,v)
for cell,s in zip(cells,syn):
    cell.set_biophysics(**parameters)
    for p,v in syn_param.items():
        setattr(s,p,v)

# Run simulation
_ = h.run()

#### Plot results
After simulation is done in previous step, run the code below to plot the results.

In [ ]:
out  = [widgets.Output() for _ in range(4)]

# Print parameters
with out[0]:
    print('Parameters:')
    for p,v in cellA.get_biophysics(**parameters).items():
        print('{} = {}'.format(p,v))
# Plot results
with out[1]:
    plt.figure(figsize=(10,4.8))
    for i,cell in enumerate(cells):
        cellname = 'Cell B' if i>0 else 'Cell A'
        clr = 'r' if i>0 else 'b'
        plt.plot(cell.t,cell.record['v'],clr,label=cellname)
    plt.xlim(cellA.t[0],cellA.t[-1])
    plt.ylim(-90,60)
    plt.legend(loc=1)
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane Voltage (mV)')
    plt.show()
for i,cell in enumerate(cells):
    with out[i+2]:
        fig,axs = cell.plot_vars(cellid=i,figsize=(6.4,8))
        plt.show()

widgets.VBox([widgets.HBox([out[0],out[1]]),widgets.HBox([out[2],out[3]])])